# 智联招聘

In [ ]:
'''
需求分析：获取职位
入口：http://sou.zhaopin.com/
'''

In [5]:
# -*- coding: GBK-*- #
from lxml import etree
import requests
import time
import re

In [6]:
headers={
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
zlzp=etree.HTML(requests.get('http://sou.zhaopin.com/',headers=headers).content.decode('utf-8'))
job_clc_url=zlzp.xpath('//div[@id="search_right_demo"]/div/div[@class="clearfixed"]/a/@href')

In [7]:
# pattern=re.compile('jl=\d+&')
# job_clc_url=['http://sou.zhaopin.com'+pattern.sub('jl=489&',i) for i in job_clc_url]
# job_clc_url

In [4]:
# #对每一类职位进行遍历
# clc=1
# for i in job_clc_url:
#     if clc==1:
#         fw=open('zlzp.txt','w',encoding='utf-8')
#     else:
#         break
        
#     r=requests.get(i,headers=headers).content.decode('utf-8')
#     job_clc_html=etree.HTML(r)
#     page=1
#     while True:
#         #获取当前页职位的连接
#         job_page_url=job_clc_html.xpath("//td[@class='zwmc']/div/a[1]/@href")
#         num=1
#         for j in job_page_url:
#             print(j)
#             try:
#                 #获取职位信息
#                 r1=requests.get(j,headers=headers).content.decode('utf-8')
#                 job_html=etree.HTML(r1)
#                 job_name=job_html.xpath("string(//div[@class='top-fixed-box'][1]/div/div/h1)")
#                 company_name=job_html.xpath("string(//div[@class='top-fixed-box'][1]/div/div/h2/a)")
#                 company_fuli=job_html.xpath("//div[@class='top-fixed-box'][1]/div/div/div/span")
#                 #print(company_fuli)
#                 job_description=job_html.xpath("string(//div[@class='terminalpage-left']/ul)")
#                 print("第{0}类第{1}页第{2}个职位".format(clc,page,num))
#                 fw.write("\n第{0}类第{1}页第{2}个职位\n".format(clc,page,num))
#                 fw.write(job_name+"\n")
#                 fw.write(company_name)
#                 for k in company_fuli:
#                     fw.write(k.xpath("text()")[0]+" ")
#                 fw.write("\n")
#                 fw.write(job_description+"\n")
#                 time.sleep(0.02)
#                 num+=1
#             except:
#                 print("erro"+"第{0}类第{1}页第{2}个职位".format(clc,page,num))
#         if job_clc_html.xpath("//a[@class='next-page']")==[]:
#             break
#         else:
#             page+=1
#             url_new=job_clc_html.xpath("//a[@class='next-page']/@href")[0]
#             r=requests.get(url_new,headers=headers).content.decode('utf-8')
#             job_clc_html=etree.HTML(r)
#     clc+=1
#     fw.close()

    
    

In [ ]:
#对象的方式
#1、获取职位分类的url列表
def get_job_cat_list(url,headers):
    r=requests.get(url,headers=headers).content.decode('utf-8')
    html=etree.HTML(r)
    job_cat_url_list=html.xpath('//div[@id="search_right_demo"]/div/div[@class="clearfixed"]/a/@href')
    pattern=re.compile('jl=\d+&')
    job_cat_url_list=['http://sou.zhaopin.com'+pattern.sub('jl=489&',i) for i in job_cat_url_list]
    return job_cat_url_list

#2、获取当前页职位url列表,并返回下一页的url
def get_job_list(url,headers):
    r=requests.get(url,headers=headers).content.decode('utf-8')
    html=etree.HTML(r)
    job_page_url=html.xpath("//td[@class='zwmc']/div/a[1]/@href")
    url_new=None
    if html.xpath("//a[@class='next-page']")==[]:
        falg=False
    else:
        url_new=html.xpath("//a[@class='next-page']/@href")[0]
        
    return [job_page_url,url_new]

#3、获取职位信息
def get_job_info(url,headers):
    data_dic={}
    r=requests.get(url,headers=headers).content.decode('utf-8')
    html=etree.HTML(r)
    data_dic['job_name']=html.xpath("string(//div[@class='top-fixed-box'][1]/div/div/h1)")
    data_dic['company_name']=html.xpath("string(//div[@class='top-fixed-box'][1]/div/div/h2/a)")
    data_dic['company_fuli']=html.xpath("//div[@class='top-fixed-box'][1]/div/div/div/span/text()")
    data_dic['company_fuli']=",".join([str(i) for i in data_dic['company_fuli']])
    data_dic['job_description']=job_html.xpath("string(//div[@class='terminalpage-left']/ul)")
    return data_dic

#数据过滤,过滤掉职位为空或者职位重复的信息
def clear_none(data,job_name_list):
    if data['job_name']==[] or data['job_name'] in job_name_list:
        return False
    else:
        job_name_list.append(data['job_name'])
        return True
    
#保存数据
def save_data(clc_num,data,filePath):
    fw=open(filePath,'a+',encoding='utf-8')
        
    for key in data.keys():
        fw.write(data[key]+"\n")
        
    fw.write("\n")
    fw.close()

In [ ]:
if __name__=="__main__":
    headers={
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/17.17134'}
    url='http://sou.zhaopin.com/'
    job_clc_url=get_job_cat_list(url,headers)#存放所有分类的职位连接
    clc=1
    job_name_list=[]
    for i in job_clc_url:
        if clc>1:
            break
            
        page=1
        url_page=i#存放当前页的连接
        while True:
            num=1
            job_page_url=get_job_list(url_page,headers)[0]#存放当前页所有职位的连接
            #print(job_page_url)
            #获取职位信息
            for j in job_page_url:
                try:
                    data=get_job_info(j,headers)
                    print("第{0}类第{1}页第{2}个职位信息".format(clc,page,num))

                    if clear_none(data,job_name_list)==True:
                        save_data(clc,data,'zlzp.txt')
                except:
                    print("第{0}类第{1}页第{2}个职位信息:erro".format(clc,page,num))
                num+=1
            
            #获取下页的连接
            if get_job_list(url_page,headers)[1]==None:
                break
            else:
                page+=1
                url_page=get_job_list(i,headers)[1]
                
            
        